# Parte 3: Machine Learning

Para esta parte del trabajo práctico elegí entrenar principalmente dos modelos: KNN y una red neuronal. \
Además, de forma adicional entrené un modelo de LightGBM para probar hacer un ensamblado con la red neuronal.

En las secciones respectivas a cada modelo doy detalles de cómo fue el proceso de entrenamiento, selección de features y demás. \
Pero antes de pasar con los modelos, incluyo una sección de Feature Engineering común a ambos modelos.

## Feature engineering común a ambos modelos

En esta sección creo features que son utilizadas por todos los modelos entrenados. \
Primero importo las funciones y objetos que necesito para esta sección y además cargo en memoria el set de entrenamiento.

Las demás importaciones las realizaré medida que los vaya utilizando.

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.experimental.enable_op_determinism()
stop_words = set(stopwords.words('english'))

train_path = 'data/train.csv'
df = pd.read_csv(train_path)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patricio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


A continuación creo las features básicas exploradas en la parte 1.

In [2]:
# Limpieza y preprocesamiento básico de texto
def clean_text(s):
    if pd.isna(s):
        return ''
    s = str(s)
    s = s.lower()
    s = re.sub(r'http\S+', ' ', s)
    s = re.sub(r'www\S+', ' ', s)
    s = re.sub(r'[^\w\s#@]', ' ', s)
    s = re.sub(r'[\s_]+', ' ', s).strip()
    filtered_words = [word for word in s.split() if word.lower() not in stop_words and not word.startswith('@')]
    filtered_words = [(w[1:] if w.startswith('#') and len(w) > 1 else w) for w in filtered_words]
    filtered_words = [w for w in filtered_words if w]
    cleaned_text = " ".join(filtered_words)
    return cleaned_text

df['text_clean'] = df['text'].apply(clean_text)
df['keyword'] = df['keyword'].fillna('no_keyword_contained')
df['location'] = df['location'].apply(clean_text)
df['location'] = df['location'].fillna('no_location_contained')

# Creación de features numéricas adicionales
df['word_count'] = df['text_clean'].apply(lambda s: len(s.split()))
df['text_len'] = df['text_clean'].apply(lambda s: sum(len(w) for w in s.split()))
df['mean_word_len'] = df.apply(lambda row: row['text_len'] / row['word_count'] if row['word_count'] > 0 else 0, axis=1)
df['num_hashtags'] = df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('#'))
df['num_mentions'] = df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('@'))
df['num_exclamations'] = df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('!'))
df['num_questions'] = df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('?'))
df['num_smilies'] = df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'[:;=8][\-o\*\']?[\)\]\(\[dDpP/:\}\{@\|\\]', s)))
df['num_repeated_chars'] = df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'(.)\1{2,}', s)))
df['num_capital_words'] = df['text'].apply(lambda s: sum(1 for w in str(s).split() if w.isupper())).fillna(0)
df['numeric_chars_count'] = df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'\d', s)))
df['stopword_ratio'] = df.apply(lambda row: sum(1 for w in str(row['text']).split() if w.lower() in stop_words) / row['word_count'] if row['word_count'] > 0 else 0, axis=1)
df['unique_word_ratio'] = df.apply(lambda row: len(set([w.strip() for w in row['text_clean'].split()])) / row['word_count'] if row['word_count'] > 0 else 0, axis=1)

# Creación de features booleanas adicionales
df['has_url'] = df['text'].apply(lambda s: 0 if pd.isna(s) else (1 if 'http' in s or 'www.' in s else 0))
df['has_hashtag'] = df['num_hashtags'].apply(lambda x: 1 if x > 0 else 0)
df['has_mention'] = df['num_mentions'].apply(lambda x: 1 if x > 0 else 0)
df['location_mentioned'] = df.apply(lambda row: 1 if row['location'].lower() in row['text_clean'] else 0, axis=1)

disaster_terms = df['keyword'].dropna().unique().tolist()
def count_terms(s, terms=disaster_terms):
    s = s.lower()
    cnt = 0
    for t in terms:
        if t in s:
            cnt += 1
    return cnt

# Feature engineering adicional 
df['disaster_terms_count'] = df['text_clean'].apply(count_terms)

Incluyo también columnas de análisis de sentimiento de los tweets

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def vader_scores(text):
    if not isinstance(text, str) or text.strip() == "":
        return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
    return analyzer.polarity_scores(text)

scores = [vader_scores(t) for t in df['text'].astype(str).tolist()]
scores_df = pd.DataFrame(scores)

df = pd.concat([df.reset_index(drop=True), scores_df.reset_index(drop=True)], axis=1)

Y por último la feature de mayor importancia para todos los modelos, los embeddings.

Si bien no está explícito en este notebook, puedo decir con certeza que son los más importantes ya que al probar retirarlos el puntaje F1 de todos los modelos disminuye drásticamente (por lo menos 0.3 puntos).

Se probó utilizar diferentes embeddings. Entre ellos Word2Vec (`GoogleNews-vectors-negative300.bin`), GloVe (`glove.twitter.27B.zip`), Fasttext (`cc.en.300.bin`) y demás configuraciones integrando pesos con TF-IDF.

En la versión final de todos los modelos utilizo Sentence Transformers (también conocidos como SBERT).\
Esto se debe a que los demás embeddings que probé están ideados para codificar palabras, por lo que debía ingeniar y probar manualmente diferentes mecanismos para transformar esos vectores de palabras en un único vector para el tweet. \
El propósito de SBERT es codificar directamente oraciones ("sentences"), por lo que consideré que llegaría a mejores resultados utilizándolo.

Si bien no incluyo ninguna prueba de resultados de los otros embeddings en este notebook, cabe mencionar que los resultados obtenidos fueron bastante similares, aunque ligeramente mejores utilizando SBERT.

In [4]:
def clean_text_minimal(s):
    s = str(s).lower()
    s = re.sub(r'http\S+', ' ', s)
    return s.strip()
df['text_emb'] = df['text'].apply(clean_text_minimal)

from sentence_transformers import SentenceTransformer
model_name = "all-MiniLM-L6-v2"
sbert = SentenceTransformer(model_name)
df_sbert = sbert.encode(df['text_emb'].astype(str).tolist(), convert_to_numpy=True)
sbert_columns = [f'sbert_{i}' for i in range(df_sbert.shape[1])]
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(df_sbert, columns=sbert_columns)], axis=1)

c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Notar que no utilizo las columna `text_clean` previamente calculada, ya que en ella se removieron las stopwords. \
Como SBERT (y la mayoría de los embeddings) son modelos contextuaes, es decir, infieren el significado de las palabras a partir del contexto, dependen fuertemente de la estructura del texto para entender el significado. Como muchas stopwords pueden ser muy relevantes para entender el sentido de los tweets ("not", "but", etc.) es crucial conservarlas para realizar los embeddings.

#### ¿cómo conviene elegir los datos de validación respecto de los de train?

Aquí divido el set original en entrenamiento y validación.

La división debe hacerse de manera tal que el set de validación sea estadísticamente idéntico al set de test ya que este último es lo más parecido que tenemos al "mundo real".

Esta división se hace de forma aleatoria (con una seed para garantizar reproducibilidad) y estratificada utilizando el target. Esto garantiza que se mantienga la misma proporción de clases en train y validación. Como a priori no tenemos los targets del set de test, la manera más fiel que tenemos de representar la "realidad" en el test de validación es basándonos en el set de entrenamiento.

En todos los modelos, utilizo `X_train` para búsqueda de hiperparámetros utilizando CrossValidation. \
Luego, entreno el modelo final con los hiperparámetros encontrados utilizando la totalidad de `X_train` y validando con `X_val` para obtener el puntaje F1 solicitado en el enunciado.


In [5]:
from sklearn.model_selection import train_test_split
X = df
y = df['target'].astype(int).values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=SEED
)

#### Features descartadas
Aprovecho el final de esta sección para nombrar algunas features o estrategias que probé y acabé descartando en el proceso porque no dieron buenos resultados.

- **Embeddings sobre categorías**: Para intentar obtener información útil de las features categóricas, se me ocurrió reaizar embeddings sobre las mismas. Probé utilizar TF-IDF, Word2Vec y SBERT (pues hay varios keywords compuestos por dos palabras), pero obtuve resultados iguales o peores que utilizando las columnas con simples encodings (one hot y mean encoding). \
Decidí descartar esta estrategia porque además de no mejorar los resultados me pareció que compejizaba el modelo innecesariamente.

- **Clustering de categorías**: Para reducir la cardinalidad de las feature `keyword`, se me ocurrió que podría agrupar palabras "similares" bajo una misma categoría. Hice esto utilizando los embeddings mencionados previamente y agrupándolos con un algoritmo de clustering. Supuse que de esta forma podría dar una vista más genérica al modelo y quizás podría resultar beneficiosa. Utilicé HDBSCAN con diferentes parámetros, y los resultados tuvieron sentido. \
Por ejemplo se encontraban juntas las keywords que tienen que ver con "destrucción" ('destroy', 'destroyed', 'destruction'), a diferencia de las relacionadas al fuego ('fire','burned','burning','burning buildings'). Luego, dependiendo de los parámetros elegidos también podían encontrarse asociaciones más o menos generales, por ejemplo, con el valor por defecto de `min_cluster_size` no se agrupaban las palabras que tenían que ver con explosiones, bombas o detonaciones, sin embargo aumentando el valor sí que se encontraban en un mismo cluster. \
Dejando de lado los resultados interesantes de cómo se formaban los clusters, el utilizar esta feature tampoco dio mejores resultados que usando las `keywords` con mean encoding. Creo que puede deberse a que esto hacía muy general la información proporcionada, lo cual dificulta la distinción de tweets sobre desastres reales. \
Un ejemplo simple que se me ocurre es que alguien podría usar una expresión como "es la bomba" que claramente no habla de un desastre, podría agruparse en el mismo cluster que un tweet que usó "explotó" ("exploded"), u otras palabras considerablemente más específicas y que probablemente sean utilizadas en tweets que sí son desastres.

- **Reducción de Dimensiones**: Intenté utilizar los algoritmos PCA y UMAP para reducir las dimensiones de las features. \
Probé utilizarlos para reducir las dimensiones de los embeddings obtenidos de SBERT, y obtuve resultados muy similares que utilizando todas las dimensiones. Podría argumentarse que habría sido buena idea dejar una reducción de dimensiones para que el modelo no tenga tantos datos de entrada y por la posible reducción en el overfitting. Sin embargo, decidí descartar esta estrategia ya que aumentaba considerablemente el tiempo de cómputo y agregaba nuevos hiperparámetros a buscar sin mejorar notablemente los resultados. \
También probé reducir las dimensiones del set de entrenamiento completo lo cual dio resultados mucho peores. Considero que puede deberse a que es más difícil reducir dimensiones cuando la información que proporciona es tan diferente, por lo que la información resultante era muy general y no muy util para realizar las predicciones deseadas.

## Primer modelo: KNN

En la siguiente celda se definen las columnas que utiliza el modelo.

Puede verse que hay varias comentadas, esto se debe a que se obtuvieron mejores resultados sin incluir esas columnas.

Supongo que incluir estas columnas no suponían una mejora en el modelo por diversas razones: \
Algunas features contienen información redundante: \
\- `neg`, `neu` y `pos` indican el sentimiento del tweet, pero compound las resume. \
\- `has_url`, `has_hashtag` y `has_mention` sólo indican si sus respectivas columnas numéricas son mayores a cero. \
Las demás features no son redundantes, pero la información que contienen puede resultar tan baja que es indistinguible del ruido que puede haber en el entrenamiento.

Aún así, la diferencia en el puntaje F1 final del modelo varía aproximadamente en ±0.01 puntos, por lo que realmente podría simplemente tratarse de ruido a la hora de entrenar. \
Por último, cabe destacar que todo lo que expliqué ahora son suposiciones sin sustento más allá de los resultados que obtuve, por lo que para llegar a puntos más concluyentes debería hacerse un análisis más profundo.

In [6]:
num_features = [
    'text_len', 'word_count', 'mean_word_len',
    'num_hashtags', 'num_mentions', 'disaster_terms_count', 
    'neg', 'pos',
    # 'neu',
    # 'compound',
    'num_exclamations', 'num_questions', 'num_smilies', 
    # 'num_repeated_chars', 'num_capital_words',
    # 'numeric_chars_count', 'stopword_ratio', 'unique_word_ratio'
]
bool_features = [
    'has_url', 'has_hashtag', 'has_mention', 
    'location_mentioned'
]

En la siguiente celda se definen los pre procesamientos que se realizan a cada columna durante cada fold de Cross Validation (y que también deben hacerse luego para predecir datos). \
Al estar utilizando únicamente objetos de `sklearn`, este proceso se facilita mucho utiizando un pipeline. Se garantiza que en cada fold de Cross Validation se hará fit únicamente con el set de entrenamiento, previniendo fuga de datos, y luego se transformarán correctamente también los datos de validación.

Utilizo OneHotEncoder para `location` limitando el máximo de categorías a utilizar. El límite de categorías a utilizar puede buscarse como hiperparámetro, pero lo dejé fijo en 20 porque es un valor que me dio buenos resultados y acelera bastante el proceso de búsqueda con GridSearch.\
Esto es útil por varios motivos. Como el campo `location` es un texto ingresado a mano por los usuarios, en realidad puede contener cualquier cosa (por ejemplo 'hollywoodland ', 'Twitter Lockout in progress', etc.). Como en general esos textos suelen ser únicos o poco frecuentes, convenientemente se agrupan bajo una misma categoría. Por otro lado, también se reduce considerablemente la cantidad de columnas generadas por el encoding. \
Para trabajo futuro, sería conveniente hacer una reducción más inteligente de esta columna. Esto podría hacerse encontrando textos diferentes que puedan referir a la misma ubicación ('L.A.' = 'Los Angeles' = 'Los Angeles, USA'), o agrupándolo en ubicaciones más grandes ('Missisipi' -> 'USA'; 'hollywood' -> 'USA').

Para la columna de `keyword` hago un mean encoding usando el `TargetEncoder` de `sklearn.preprocessing`. Probé hacer una reducción manual de la cantidad de categorías diferentes en esta columna pero obtuve resultados peores. 

Como KNN es un modelo que se basa completamente en calcular distancia, es necesario escalar las features para que tengan dimensiones similares. \
Si no escalara las features, las features con valores más grandes dominaría por completo el cálculo de distancia. En cambio, al escalar las features todas tienen valores de una magnitud similar y contribuyen de forma más proporcional al cálculo de la distancia.

Encontré mejores resultados usando el `RobustScaler` de `sklearn.preprocessing` que usando el `StandardScaler`. Supongo que esto se debe a que el primero toma en consideración los outliers de forma más robusta, porque esa es la principal diferencia entre ambos mecanismos.



In [7]:
from sklearn.preprocessing import RobustScaler, StandardScaler, TargetEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        # ('nombre', transformador, columnas_a_aplicar)
        
        ('num', RobustScaler(), num_features),
        
        ('bool', 'passthrough', bool_features),
        
        ('loc_ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=20), ['location']),
        
        ('kyw_target', TargetEncoder(random_state=SEED), ['keyword']),

        ('sbert', RobustScaler(), sbert_columns),
    ],
    remainder='drop'
)

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsClassifier())
])

Luego de definir el pipeline, pasamos a la búsqueda de hiperparámetros y evaluación en validación del modelo.

Defino primero el espacio de hiperparámetros a buscar en `param_grid`. Aquí en realidad fui cambiando los parámetros que probé. Comencé utilizando rangos más grandes ([5,10,20,50,100,200]) y fui dando granularidad más específica según iba obteniendo resultados. En el estado final de la celda, dejé algunos parámetros que sé no darán buenos resultados para ilustrar la búsqueda. Puede notarse que hay varios valores de `n_neighbors` consecutivos entre 35 y 40, esto se debe a que en "rondas" anteriores, los mejores hiperparámetros estaban entre 30 y 40 por lo que supuse que el mejor valor estaría entre esos números.

Me parece interesante destacar una tendencia que noté durante las iteraciones: al incluir más features para que utilice el modelo, el mejor valor de `n_neighbors` tiende a ser más alto. \
Si bien esto podría ser una simple casualidad, mi hipótesis es que puede deberse a ese "ruido" provocado por incluir demasiadas features. Ese ruido genera que el modelo necesite buscar en más vecinos cercanos para obtener "suavizar" el ruido y obtener una decisión más precisa.

Luego de obtener los mejores hiperparámetros, se pasa a reentrenar el modelo con todo el set de entrenamiento (sin Cross Validation) y a obtener el valor final del puntaje f1 de validación.

In [8]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score


param_grid = {
    'model__n_neighbors': [5, 10, 20, 30, 35, 37, 38, 39, 40, 42, 50, 60, 100],
    'model__weights': ['uniform', 'distance'],
    'model__metric': ['euclidean', 'manhattan', 'cosine', 'minkowski'],
    # 'preprocessor__loc_ohe__max_categories': [10, 20, 50, 100, 200],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
grid = GridSearchCV(full_pipeline, param_grid, scoring='f1', cv=cv, n_jobs=-1, verbose=1)

grid.fit(X_train, y_train)

print("Mejores parámetros:", grid.best_params_)

Fitting 5 folds for each of 104 candidates, totalling 520 fits
Mejores parámetros: {'model__metric': 'cosine', 'model__n_neighbors': 38, 'model__weights': 'distance'}


In [9]:
best_params = grid.best_params_
best_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsClassifier(**{k.replace('model__', ''): v for k, v in grid.best_params_.items() if k.startswith('model__')}))
])

best_knn.set_params(**best_params)
best_knn.fit(X_train, y_train)

y_val_pred = best_knn.predict(X_val)
print("F1 en validación:", f1_score(y_val, y_val_pred))

F1 en validación: 0.8018433179723502


Como se observa, el puntaje obtenido es apenas mayor a 0.8. \
Este valor se obtuvo luego de refinar al máximo las features seleccionadas y el preprocesamiento de las mismas y considero que ese es el límite alcanzable utilizando estas herramientas.

Para obtener valores más altos y modelos que hagan mejores predicciones, se deberían usar herramientas más complejas y que permitan un mejor análisis de la intención detrás de los tweets.

## Segundo modelo: LightGBM

Como segundo modelo utilizo LightGBM. La estrategia empleada es muy similar a la utilizada en KNN, con pequeñas diferencias.

A diferencia que en KNN, aquí no excluyo ninguna de las features creadas. Esto es así porque el modelo busca automáticamente las más útiles. \
Se puede configurar la proporción de features tomadas en cuenta con el hiperparámetro `colsample_bytree`.

In [10]:
from lightgbm import LGBMClassifier

metadata_features = [
    'text_len', 'word_count', 'mean_word_len',
    'num_hashtags', 'num_mentions', 'disaster_terms_count', 
    'compound',
    'num_exclamations', 'num_questions', 'num_smilies', 
    'num_repeated_chars', 'num_capital_words',
    'has_url', 'has_hashtag', 'has_mention', 
    'location_mentioned'
]

preprocessor_lgbm = ColumnTransformer([
    ('sbert', 'passthrough', sbert_columns), 
    ('keyword_enc', TargetEncoder(random_state=SEED), ['keyword']),
    ('location_enc', OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=50), ['location']),
    ('meta_scaled', 'passthrough', metadata_features)
], remainder='drop')

La otra diferencia con la estrategia usada en KNN es la forma de buscar los hiperparámetros.

Esta vez utilizo otra libreria para hacerlo, `optuna`. Este framework promete hacer más óptima la búsqueda de hiperparámetros, tanto con optimizaciones internas como agregando mecanismos que pueden ser adoptados por el usuario. \

En la siguiente celda defino una función "objetivo" la cual `optuna` intentará maximizar buscando hiperparámetros dentro del rango definido. Esta función objetivo devuelve la media de todos los puntajes F1 obtenidos en los folds de cross validation, por lo que es lo que se buscará maximizar.

La librería permite añadir más optimizaciones a la búsqueda de hiperparámetros, pero no conseguí hacerlo de manera reproducible. Puede verse la adopción de una de estas estrategias en el [anexo](##Anexo)

In [11]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    # A. Sugerir Hiperparámetros
    # Optuna elige valores inteligentes dentro de estos rangos
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        
        'random_state': SEED,
        'is_unbalance': True,
        'n_jobs': 4,
        'verbose': -1,
        'force_col_wise': True
    }

    model = LGBMClassifier(**param)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor_lgbm),
        ('model', model)
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='f1', n_jobs=1)

    return scores.mean()

La librería permite también configurar finamente el tiempo empleado en la búsqueda. Esta vez utilizo n_trials para garantizar la reproducibilidad, pero también puede usarse un timeout.

In [12]:
optuna_sampler = optuna.samplers.TPESampler(seed=SEED)
lgbm_study = optuna.create_study(direction='maximize', sampler=optuna_sampler)
lgbm_study.optimize(objective, n_trials=10, n_jobs=1)

print("Mejores Hiperparámetros:")
print(lgbm_study.best_params)

[I 2025-11-20 03:08:57,584] A new study created in memory with name: no-name-2bd9452f-5793-4a4d-8802-49fa43c1a59d
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-

Mejores Hiperparámetros:
{'n_estimators': 973, 'num_leaves': 63, 'max_depth': 3, 'learning_rate': 0.015746438450976667, 'subsample': 0.6625916610133735, 'colsample_bytree': 0.864803089169032, 'reg_alpha': 0.005470376807480391, 'reg_lambda': 0.9658611176861268, 'min_child_samples': 50}


Para entrenar la versión final del modelo utilizo estos parámetros hardcodeados que encontré en una búsqueda anterior, pero perdí al volver a correr la celda anterior con parámetros diferentes.

In [13]:
# best_params = lgbm_study.best_params
best_params = {'n_estimators': 940, 'num_leaves': 50, 'max_depth': 10, 'learning_rate': 0.018865211249690032, 'subsample': 0.8480621510679667, 'colsample_bytree': 0.6216966666839611, 'reg_alpha': 7.915284888606708e-08, 'reg_lambda': 0.008594814828760103, 'min_child_samples': 21}
best_params.update({
    'verbose': -1,
    # necesarios para asegurar la reproducibilidad:
    'random_state': SEED,
    'is_unbalance': True,
    'n_jobs': 1, 
    'force_col_wise': True
})

final_lgbm = LGBMClassifier(**best_params)

final_pipeline = Pipeline([
    ('preprocessor', preprocessor_lgbm),
    ('model', final_lgbm)
])

final_pipeline.fit(X_train, y_train)

preds_lgbm_optuna = final_pipeline.predict(X_val)

f1_final_optuna = f1_score(y_val, preds_lgbm_optuna)
print(f"F1 Score Final en X_val: {f1_final_optuna:.5f}")

F1 Score Final en X_val: 0.80671


c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


## Para el mejor modelo de ambos, ¿cuál es el score en la competencia? 

Ya que obtuve un puntaje muy similar entre ambos modelos, decidí utilizar el de KNN para predecir los datos de test. \
Esto se debe a que KNN es un modelo más simple por lo que, siguiendo el principio de la Navaja de Ockham, podría considerarse un mejor modelo.

In [10]:
test_path = 'data/test.csv'
test_df = pd.read_csv(test_path)

test_df['text_clean'] = test_df['text'].apply(clean_text)
test_df['keyword'] = test_df['keyword'].fillna('no_keyword_contained')
test_df['location'] = test_df['location'].apply(clean_text)
test_df['location'] = test_df['location'].fillna('no_location_contained')
test_df['word_count'] = test_df['text_clean'].apply(lambda s: len(s.split()))
test_df['text_len'] = test_df['text_clean'].apply(lambda s: sum(len(w) for w in s.split()))
test_df['mean_word_len'] = test_df.apply(lambda row: row['text_len'] / row['word_count'] if row['word_count'] > 0 else 0, axis=1)
test_df['num_hashtags'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('#'))
test_df['num_mentions'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('@'))
test_df['num_exclamations'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('!'))
test_df['num_questions'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else s.count('?'))
test_df['num_smilies'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'[:;=8][\-o\*\']?[\)\]\(\[dDpP/:\}\{@\|\\]', s)))
test_df['num_repeated_chars'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'(.)\1{2,}', s)))
test_df['num_capital_words'] = test_df['text'].apply(lambda s: sum(1 for w in str(s).split() if w.isupper())).fillna(0)
test_df['numeric_chars_count'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else len(re.findall(r'\d', s)))
test_df['stopword_ratio'] = test_df.apply(lambda row: sum(1 for w in str(row['text']).split() if w.lower() in stop_words) / row['word_count'] if row['word_count'] > 0 else 0, axis=1)
test_df['unique_word_ratio'] = test_df.apply(lambda row: len(set([w.strip() for w in row['text_clean'].split()])) / row['word_count'] if row['word_count'] > 0 else 0, axis=1)
test_df['has_url'] = test_df['text'].apply(lambda s: 0 if pd.isna(s) else (1 if 'http' in s or 'www.' in s else 0))
test_df['has_hashtag'] = test_df['num_hashtags'].apply(lambda x: 1 if x > 0 else 0)
test_df['has_mention'] = test_df['num_mentions'].apply(lambda x: 1 if x > 0 else 0)
test_df['location_mentioned'] = test_df.apply(lambda row: 1 if row['location'].lower() in row['text_clean'] else 0, axis=1)
test_df['disaster_terms_count'] = test_df['text_clean'].apply(count_terms)

scores = [vader_scores(t) for t in test_df['text'].astype(str).tolist()]
scores_test_df = pd.DataFrame(scores)
test_df = pd.concat([test_df.reset_index(drop=True), scores_test_df.reset_index(drop=True)], axis=1)

test_df['text_emb'] = test_df['text'].apply(clean_text_minimal)
test_df_sbert = sbert.encode(test_df['text_emb'].astype(str).tolist(), convert_to_numpy=True)
sbert_columns = [f'sbert_{i}' for i in range(test_df_sbert.shape[1])]
test_df = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(test_df_sbert, columns=sbert_columns)], axis=1)

In [11]:
knn_test_preds = best_knn.predict(test_df)
knn_result = pd.DataFrame({
    'id': test_df['id'],
    'target': knn_test_preds,
})
knn_result.to_csv('knn_test_predictions.csv', index=False)

Subiendo el archivo resultante a la competencia, obtuve un puntaje F1 de 0.81826.\
Es un poco mayor al obtenido en validación (0.80184), lo cual puede deberse a que el set de test es estadísticamente diferente al de validación, o simplemente a variaciones normales en el desempeño del modelo.

## Anexo

Me resultó muy difícil obtener puntajes mayores a 0.8 para esta parte. \
Obtener 0.79 fue sencillo, de hecho, como se verá en la parte 4, eso puede alcanzarse fácilmente utilizando simplemente KNN y embeddings con SBERT. \
Pero pasar ese umbral y conseguir un modelo superador resultó extremadamente costoso. Intenté varios enfoques distintos, diferentes modelos, diferentes features agregadas, etc. 

Además de las features adicionales mencionadas al [final de la sección Feature engineering](####Features-descartadas), probé algunas estrategias que describo en esta sección.

### Modelo Extra: Red Neuronal
Originalmente este iba a ser mi segundo modelo, pero no logré obtener una puntuación mayor a 0.8 de manera consistente.

Intenté utilizar diferentes enfoques, entre ellos diferentes ramas de input de datos, usar una layer LSTM, convoluciones, usar embeddings preentrenados y refinarlos con los datos de entrenamiento, etc.
Dejo adjunta la última versión que estuve intentando mejorar, que es una que tiene 3 ramas de input de datos.

In [ ]:
num_features = [
    'text_len', 'word_count', 'mean_word_len',
    'num_hashtags', 'num_mentions', 'disaster_terms_count', 
    # 'neg', 'neu', 'pos',
    'compound',
    'num_exclamations', 'num_questions', 'num_smilies', 
    'num_repeated_chars', 'num_capital_words',
    # 'numeric_chars_count', 'stopword_ratio', 'unique_word_ratio'
]
bool_features = [
    # 'has_url', 'has_hashtag', 'has_mention', 
    # 'location_mentioned'
]

A continuación se encuentra la definición en sí de la red neuronal.

Primeramente se puede observar que se reciben hiperparámetros y las dimensiones de los datos de entrada.

Se pueden diferenciar distintos tipos de capas, entre ellas:
- **Input** define datos de entrada. Debe especificarse la dimensión de la entrada, el tipo de dato y un nombre para poder referenciar los datos que se envíen luego al modelo.
- **Dense** es una capa común, densamente conectada. Puede definirse cantidad de neuronas, función de activación, si usar o no un bias, entre otros. Notar también que se especifica un inicializador para los parámetros para intentar asegurar la reproducibilidad.
- **Dropout** es una capa de regularización. Simplemente desactiva algunas conexiones entre neuronas con cierta probabilidad. Esto hace que el entrenamiento sea más robusto y que las predicciones no dependan únicamente de un conjunto específico de features o neuronas. Aquí también se explicita la semilla para intentar asegurar la reproducibilidad.
- **Batch Normalization** esta es una capa que no vimos en clase y me pareció interesante. Normaliza los datos del batch entrante para pasar datos con una distribución normal a la siguiente capa. Esto facilita el entrenamiento de las diferentes capas, ya que hace que los datos recibidos no tengan escalas dependientes de la capa anterior. \
Para utilizar estas capas la práctica más utilizada es desactivar la activación en la capa densa previa y utilizar la función de activación luego de la normalización. Desconozco exactamente por qué se hace así, pero parece ser la forma en que suele ser implementado y resultó dar buenos resultados.

También se puede notar que la red está definida con tres ramas que reciben diferentes datos de entrada. Esto permite al modelo dar un trato diferente a los diferentes tipos de datos. \
Por un lado se reciben los embeddings de SBERT. Esta rama tiene dos capas densas intercaladas con capas de Dropout para regularización. Es notable el declive del resultado cuando no se incluyen las capas de Dropout. \
En otra rama se recibe el input categórico, en donde se recibe tanto la categoría `keyword` con mean encoding como `location` con one hot. La estructura de capas aempleada aquí es la misma que para el input de SBERT, pero con sus propio hiperparámetro para la cantidad de neuronas. \
Finalmente, la rama de input numérico recibe las demás features, tanto numéricas como booleanas (aunque como vimos antes no se incluyen variables booleanas en este modelo). Aquí adoptamos por primera vez el patrón para usar Batch Normalization.

Probé diferentes distribuciones para las ramas (por ejemplo, `keyword` que es mean encoding pasarlo junto a las numéricas y `location` que es one hot junto a las booleanas) pero esta fue la que dio ligeramente mejores resultados.

Luego todas las ramas de input se combinan en un último tramo en el que se reduce la cantidad de neuronas por capa hasta llegar a una última neurona de output con una función de activación sigmoidea, la cual dará el resultado de la predicción.

In [ ]:
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras import initializers
import keras
from keras import layers

def build_neural_network(sbert_dim, cat_dim, num_dim, hparams={}):
    
    SBERT_UNITS = hparams.get('sbert_units', sbert_dim)
    CAT_UNITS = hparams.get('cat_units', cat_dim)
    NUM_UNITS = hparams.get('num_units', num_dim)
    FINAL_UNITS = hparams.get('final_units', 64)
    DROPOUT_RATE = hparams.get('dropout_rate', 0.5)
    LEARNING_RATE = hparams.get('learning_rate', 0.001)

    init = initializers.GlorotUniform(seed=SEED)

    # INPUT SBERT (EMBEDDING)
    input_sbert = keras.Input(shape=(sbert_dim,), dtype="float32", name="input_sbert")
    x_sbert = layers.Dense(SBERT_UNITS, activation='relu', kernel_initializer=init)(input_sbert)
    x_sbert = layers.Dropout(DROPOUT_RATE, seed=SEED)(x_sbert)
    x_sbert = layers.Dense(SBERT_UNITS//2, activation='relu', kernel_initializer=init)(x_sbert)
    x_sbert = layers.Dropout(DROPOUT_RATE, seed=SEED)(x_sbert)

    # INPUT CATEGÓRICO
    input_cat = keras.Input(shape=(cat_dim,), dtype="float32", name="input_cat")
    x_cat = layers.Dense(CAT_UNITS, activation='relu', kernel_initializer=init)(input_cat)
    x_cat = layers.Dropout(DROPOUT_RATE, seed=SEED)(x_cat)
    x_cat = layers.Dense(CAT_UNITS//2, activation='relu', kernel_initializer=init)(x_cat)
    x_cat = layers.Dropout(DROPOUT_RATE, seed=SEED)(x_cat)

    # INPUT NUMERICO Y BOOLEANO
    input_num = keras.Input(shape=(num_dim,), dtype="float32", name="input_num")
    x_num = layers.Dense(NUM_UNITS, use_bias=False, kernel_initializer=init)(input_num)
    x_num = layers.BatchNormalization()(x_num)
    x_num = layers.Activation('relu')(x_num)
    x_num = layers.Dropout(DROPOUT_RATE, seed=SEED)(x_num)
    
    # COMBINACIÓN DE RAMAS Y OUTPUT
    merged = layers.concatenate([x_sbert, x_cat, x_num])
    final_output = layers.Dense(FINAL_UNITS, use_bias=False, kernel_initializer=init)(merged)
    final_output = layers.BatchNormalization()(final_output)
    final_output = layers.Activation('relu')(final_output)
    final_output = layers.Dropout(DROPOUT_RATE, seed=SEED)(final_output)
    final_output = layers.Dense(FINAL_UNITS//2, use_bias=False, kernel_initializer=init)(final_output)
    final_output = layers.Activation('relu')(final_output)
    # final_output = layers.Dropout(DROPOUT_RATE, seed=SEED, kernel_initializer=init)(final_output)
    preds = layers.Dense(1, activation='sigmoid')(final_output)
    
    model = keras.Model(inputs=[input_sbert, input_cat, input_num], outputs=preds)

    model.compile(
        loss="binary_crossentropy",
        optimizer=RMSprop(learning_rate=LEARNING_RATE), 
        metrics=["AUC", F1Score(threshold=0.5, name="f1_score")]
    )
    return model

Utilizo una función de poda de `optuna` (`TFKerasPruningCallback`), que permite cortar prematuramente el entrenamiento de un modelo si detecta que no supondrá una mejora al mejor ya encontrado. Esto puede resultar muy útil a la hora de buscar hiperparámetros, pero viene con la contra de tener que implementar un poco más a mano el entrenamiento de cross validation, ya que no es completamente compatible.

En la función objetivo entreno la red neuronal usando cross validation y el set de entrenamiento dividido previamente, dejando el set de validación para la comparación con otros modelos. \
Notar que en la función `train_neural_network_model` definí "manualmente" el preprocesamiento de datos y el entrenamiento de los diferentes folds del cross validation, cosa que en el modelo anterior se podía hacer de manera "automática" con un Pipeline y otros objetos de `sklearn`. En este proceso tuve que ser sumamente cuidadoso de no cometer data leaks, ya que suponen un gran problema para el entrenamiento del modelo, dando acceso a datos de validación durante el entrenamiento y permitiéndole "hacer trampa".

También utilizo un `EarlyStopper` callback para evitar overfitting. El mismo detiene el entrenamiento si las métricas de validación dejan de mejorar a pesar que las métricas de entrenamiento lo sigan haciendo. \
Para entrenar la red neuronal, configuré la métrica AUC-ROC como métrica a verificar por el `EarlyStopper` y el callback de poda de `optuna`. Si bien no hay tanta diferencia con entrenar usando F1, decidí entrenar la red utilizando la curva AUC ya que parece ser lo más estándar y utilizado ampliamente.

Finalmente, es importante devolver los encoders y scalers que se utilizaron durante el entrenamiento ya que son los que se deben utilizar para dar nuevos inputs a predecir para el modelo.

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from optuna.integration import TFKerasPruningCallback

def objective(trial):
    hparams = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'sbert_units': trial.suggest_int('sbert_units', 256, 720, step=8),
        'cat_units': trial.suggest_int('cat_units', 4, 86, step=2),
        'num_units': trial.suggest_int('num_units', 4, 86, step=2),
        'final_units': trial.suggest_int('final_units', 16, 256, step=4),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.6),
    }
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)
    fold_f1_scores = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
        
        model, x_val_formated, y_val_formated, _, _, _ = \
            train_neural_network_model(hparams, X_train.iloc[train_idx], X_train.iloc[val_idx], y_train[train_idx], y_train[val_idx], trial)

        # Evaluar...
        scores = model.evaluate(
            x=x_val_formated,
            y=y_val_formated,
            verbose=0
        )

        val_f1 = scores[2]  # indice 2 = 'f1_score'
        fold_f1_scores.append(val_f1)

    # 1c. Devuelve el score promedio de este "trial"
    mean_f1 = np.mean(fold_f1_scores)
    
    # Reporta si el trial fue podado
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
        
    return mean_f1

def train_neural_network_model(hparams, x_train, x_val, y_train, y_val, trial=None):
    y_train_fold = y_train.reshape(-1, 1)
    y_val_fold = y_val.reshape(-1, 1)

    # embedding input    
    X_sbert_train = x_train[sbert_columns]
    X_sbert_val = x_val[sbert_columns]

    oh_encoder = OneHotEncoder(handle_unknown='ignore', max_categories=20, sparse_output=False)
    mean_encoder = TargetEncoder(random_state=SEED)
    
    # categorical input
    # onehot para location + target encoding para keywords
    X_cat_train = np.hstack([
        mean_encoder.fit_transform(x_train[['keyword']], y_train_fold),
        oh_encoder.fit_transform(x_train[['location']])
    ])
    X_cat_val = np.hstack([
        mean_encoder.transform(x_val[['keyword']]),
        oh_encoder.transform(x_val[['location']])
    ])

    numeric_transformer = StandardScaler()
    X_num_train = numeric_transformer.fit_transform(np.hstack([x_train[num_features], x_train[bool_features]]))
    X_num_val = numeric_transformer.transform(np.hstack([x_val[num_features], x_val[bool_features]]))
        
    model = build_neural_network(
            sbert_dim=X_sbert_train.shape[1],
            cat_dim=X_cat_train.shape[1],
            num_dim=X_num_train.shape[1],
            hparams=hparams
        )
        
    early_stopper = EarlyStopping(monitor="AUC", mode="max", patience=5, restore_best_weights=True)
    
    if trial is not None:
        # poda de Optuna
        pruning_callback = TFKerasPruningCallback(trial, "AUC")

    validation_data = (
        {'input_sbert': X_sbert_val, 'input_cat': X_cat_val, 'input_num': X_num_val},
        y_val_fold
    )

    model.fit(
        x={'input_sbert': X_sbert_train, 'input_cat': X_cat_train, 'input_num': X_num_train},
        y=y_train_fold,
        batch_size=128,
        epochs=30,
        validation_data=validation_data,
        callbacks=[early_stopper, pruning_callback] if trial is not None else [early_stopper],
        verbose=0
    )

    return model, validation_data[0], validation_data[1], mean_encoder, oh_encoder, numeric_transformer

En la siguiente celda comienzo la búsqueda de hiperparámetros. Esta búsqueda es muy personalizabe y permite decidir fácilmente cuánto tiempo invertir en la misma.

In [ ]:
import optuna.samplers

optuna_sampler = optuna.samplers.TPESampler(seed=SEED)
nn_study = optuna.create_study(direction="maximize", sampler=optuna_sampler) 

nn_study.optimize(objective, timeout=120)

print(f"Mejor F1 Score: {nn_study.best_value:.4f}")
print("Mejores Hiperparámetros:")
print(nn_study.best_params)

[I 2025-11-20 00:27:57,924] A new study created in memory with name: no-name-f4770dca-0988-47a1-b29f-56b0cd31a796
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:503: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:503: UserWarning: The reported value is igno

Mejor F1 Score: 0.7566
Mejores Hiperparámetros:
{'learning_rate': 0.00031489116479568613, 'sbert_units': 704, 'cat_units': 64, 'num_units': 54, 'final_units': 52, 'dropout_rate': 0.17799726016810133}


Un detalle importante de este modelo es que, a pesar de mis mayores esfuerzos por establecer una semilla y utilizar todas las funciones que encontré, no logré hacer que sea reproducible. Cada vez que vuelvo a entrenar el modelo con los mismos parámetros obtengo un resultado diferente de score F1.

Aún así, los máximos resultados que obtuve no superaron 0.81 de puntaje F1 en validación.

In [ ]:
nn_x_train, nn_x_val, nn_y_train, nn_y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED
)
best_nn, _, _, mean_encoder, oh_encoder, numeric_transformer = \
    train_neural_network_model(
        nn_study.best_params,
        nn_x_train,
        nn_x_val,
        nn_y_train,
        nn_y_val
    )

# Validación final
scores = best_nn.evaluate(
    x={
        'input_sbert': X_val[sbert_columns],
        'input_cat': np.hstack([
            mean_encoder.transform(X_val[['keyword']]),
            oh_encoder.transform(X_val[['location']])
        ]),
        'input_num': numeric_transformer.transform(
            np.hstack([X_val[num_features], X_val[bool_features]])
        )
    },
    y=y_val.reshape(-1, 1),
    verbose=0
)

f1_nn = scores[2]  # indice 2 = 'f1_score'
print(f"Mejor F1 en validación: {f1_nn:.4f}")

c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mejor F1 en validación: 0.7846


### Blending
Otra estrategia que intenté utilizar fue combinar las predicciones de dos modelos.

Combinando la red neuronal con LightGBM se obtienen ligeramente mejores puntajes F1. Si bien esto podría ser ruido, también es razonable pensar que los modelos se complementan bien: la red neuronal podría estar, por ejemplo, haciendo predicciones más certeras basándose en los embeddings mientras que LGBM podría ser mejor analizando los metadatos. De esta forma se podrían obtener resultados superadores, pero aún así no conseguí que sean mucho mejores.

In [ ]:
preds_lgbm = final_pipeline.predict_proba(X_val)[:, 1]
preds_nn = best_nn.predict(x={
        'input_sbert': X_val[sbert_columns],
        'input_cat': np.hstack([
            mean_encoder.transform(X_val[['keyword']]),
            oh_encoder.transform(X_val[['location']])
        ]),
        'input_num': numeric_transformer.transform(np.hstack([
            X_val[num_features],
            X_val[bool_features]
        ]))
    }
)

best_f1 = 0
best_weight = 0

for nn_weight in np.arange(0.1, 1.0, 0.1):
    lgbm_weight = 1.0 - nn_weight
    
    # (weight_nn * preds_nn) + (weight_lgbm * preds_lgbm)
    blend_preds = (preds_nn.reshape(-1) * nn_weight) + (preds_lgbm * lgbm_weight)
    
    final_preds = (blend_preds > 0.5).astype(int)
    current_f1 = f1_score(y_val, final_preds)
    
    print(f"Peso NN: {nn_weight:.1f} | Peso LGBM: {lgbm_weight:.1f} | F1: {current_f1:.4f}")
    
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_weight = nn_weight

print("\n--- Mejor Resultado ---")
print(f"Mejor F1 solo LGBM: {f1_final_optuna:.4f}")
print(f"Mejor F1 solo NN: {f1_nn:.4f}")
print(f"Mejor F1 de Blending: {best_f1:.4f}")
print(f"Mejor Peso (para NN): {best_weight:.1f}")
print(f"Mejor Peso (para LGBM): {1.0-best_weight:.1f}")


--- Buscando el Peso Óptimo ---
 1/48 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step

c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Peso NN: 0.1 | Peso LGBM: 0.9 | F1: 0.7920
Peso NN: 0.2 | Peso LGBM: 0.8 | F1: 0.7975
Peso NN: 0.3 | Peso LGBM: 0.7 | F1: 0.7936
Peso NN: 0.4 | Peso LGBM: 0.6 | F1: 0.7920
Peso NN: 0.5 | Peso LGBM: 0.5 | F1: 0.7903
Peso NN: 0.6 | Peso LGBM: 0.4 | F1: 0.7885
Peso NN: 0.7 | Peso LGBM: 0.3 | F1: 0.7876
Peso NN: 0.8 | Peso LGBM: 0.2 | F1: 0.7867
Peso NN: 0.9 | Peso LGBM: 0.1 | F1: 0.7846

--- Mejor Resultado ---
Mejor F1 solo LGBM: 0.7914
Mejor F1 solo NN: 0.7846
Mejor F1 de Blending: 0.7975
Mejor Peso (para NN): 0.2
Mejor Peso (para LGBM): 0.8


### Fine-tuning de un modelo preentrenado (distilbert)

Aunque por muy poco, este fue modelo que mejor puntaje obtuvo. Se trata de un modelo preentrenado de HuggingFace, al cual le hice fine-tunning utilizando el set de entrenamiento para que pueda predecir el target. \
Aún utilizando un transformer complejo con fine-tunning, el puntaje obtenido es apenas 0.81. Esto habla de la complejidad del problema. Seguramente haya que dar con algún dato clave en particular para lograr obtener mejores puntajes.

In [2]:
import evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
MODEL_CHECKPOINT = "distilbert-base-uncased"

c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.model_selection import train_test_split


df = pd.read_csv('data/train.csv')

df['text'] = df['text'].fillna("")
df = df.rename(columns={'target': 'label'})

df_train, df_val = train_test_split(
    df,
    test_size=0.2,
    random_state=SEED,
    stratify=df['label']
)

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_val = dataset_val.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["id", "keyword", "location", "text", "__index_level_0__"])
tokenized_val = tokenized_val.remove_columns(["id", "keyword", "location", "text", "__index_level_0__"])

Map: 100%|██████████| 1523/1523 [00:00<00:00, 17176.97 examples/s]


In [5]:
labels = ["Not Disaster", "Disaster"]
id2label = {0: "Not Disaster", 1: "Disaster"}
label2id = {"Not Disaster": 0, "Disaster": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    predictions = np.argmax(logits, axis=-1)
    
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    acc = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    
    return {
        "f1": f1,
        "accuracy": acc
    }

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    
    learning_rate=2e-5,
    weight_decay=0.01,
    
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\Patricio\AppData\Local\Temp\ipykernel_30620\2050177316.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.366562,0.818033,0.854235
2,0.406200,0.420679,0.807202,0.831254
3,0.283200,0.431146,0.811866,0.841760


c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1143, training_loss=0.33385882707197, metrics={'train_runtime': 2621.0148, 'train_samples_per_second': 6.971, 'train_steps_per_second': 0.436, 'total_flos': 605044843361280.0, 'train_loss': 0.33385882707197, 'epoch': 3.0})

In [11]:
eval_results = trainer.evaluate()

print(f"Puntaje F1 en validación: {eval_results['eval_f1']:.4f}")

c:\Users\Patricio\miniconda3\envs\ml_env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Puntaje F1 en validación: 0.8180
